# DfLib with Glue
This notebook demonstrates how our minimal glue library makes common operations easier while still allowing us to use the full DFLib API when needed.

In [1]:
// First cell: Import dependencies
%maven org.dflib:dflib-jupyter:2.0.0-M2
%maven org.dflib:dflib-csv:2.0.0-M2
// Our glue library should already be in the classpath from the local Maven install

In [2]:
// Import the libraries
import org.jjavaglue.dflib.DS;
import org.dflib.*;
import java.util.*;

In [3]:
// Create sample data
Map<String, List<?>> programmingLanguages = new HashMap<>();
programmingLanguages.put("name", Arrays.asList("Java", "Python", "JavaScript", "C#", "Go"));
programmingLanguages.put("popularity", Arrays.asList(16.8, 36.2, 9.2, 6.3, 4.2));
programmingLanguages.put("year", Arrays.asList(1995, 1991, 1995, 2000, 2009));

In [4]:
// Create DataFrame using our glue library
DataFrame df = DS.fromMap(programmingLanguages);

// Display information about the DataFrame
DS.describe(df);

DataFrame: 3 columns × 5 rows

Columns:
  year                 ArraySeries          Sample: [1995, 1991, 1995]
  popularity           ArraySeries          Sample: [16.8, 36.2, 9.2]
  name                 ArraySeries          Sample: [Java, Python, JavaScript]

First 5 rows:

year popularity name      
---- ---------- ----------
1995       16.8 Java      
1991       36.2 Python    
1995        9.2 JavaScript
2000        6.3 C#        
2009        4.2 Go        
5 rows x 3 columns

Numeric Summaries:


In [5]:
// Sort by popularity (descending)
DataFrame sortedByPopularity = DS.sort(df, "popularity", false);
System.out.println("Languages sorted by popularity (descending):");
DS.show(sortedByPopularity);

Languages sorted by popularity (descending):

year popularity name      
---- ---------- ----------
1991       36.2 Python    
1995       16.8 Java      
1995        9.2 JavaScript
2000        6.3 C#        
2009        4.2 Go        
5 rows x 3 columns


In [6]:
// Sort by year (ascending, which is the default)
DataFrame sortedByYear = DS.sort(df, "year");
System.out.println("Languages sorted by year (ascending):");
DS.show(sortedByYear);

Languages sorted by year (ascending):

year popularity name      
---- ---------- ----------
1991       36.2 Python    
1995       16.8 Java      
1995        9.2 JavaScript
2000        6.3 C#        
2009        4.2 Go        
5 rows x 3 columns


In [10]:
// Let's make some calculations without the glue library
// Calculate percentage of total popularity
double totalPopularity = 0;
Series<?> popularitySeries = df.getColumn("popularity");
for (int i = 0; i < popularitySeries.size(); i++) {
    Object value = popularitySeries.get(i);
    if (value instanceof Number) {
        totalPopularity += ((Number) value).doubleValue();
    }
}

// Calculate percentages and create a new DataFrame column by column
double finalTotal = totalPopularity; // Need final for lambda

// Create a new DataFrame with all existing columns plus the percentage column
String[] columns = new String[df.width() + 1];
Series<?>[] series = new Series[df.width() + 1];

// Copy existing columns
for (int i = 0; i < df.width(); i++) {
    String colName = df.getColumnsIndex().get(i);
    columns[i] = colName;
    series[i] = df.getColumn(colName);
}

// Add the new percentage column
columns[df.width()] = "percentage";
series[df.width()] = df.getColumn("popularity").map(v -> {
    double popularity = ((Number) v).doubleValue();
    return (popularity / finalTotal) * 100;
});

// Create the new DataFrame
DataFrame withPercentage = DataFrame.byColumn(columns).of(series);

System.out.println("Languages with percentage of total popularity:");
DS.show(withPercentage);

Languages with percentage of total popularity:

year popularity name               percentage
---- ---------- ---------- ------------------
1995       16.8 Java       23.108665749656122
1991       36.2 Python      49.79367262723521
1995        9.2 JavaScript 12.654745529573589
2000        6.3 C#          8.665749656121045
2009        4.2 Go          5.777166437414031
5 rows x 4 columns
